In [146]:
# imports
import cv2
import os
import numpy as np
import math
from dataclasses import dataclass,fields
from utils import *

In [147]:
# TODO:convert_video_to_frames
"""
params : video name 
return : create output directory include the frames of this video 
"""

def convert_video_to_frames(video_name):
    # Set the video file path
    video_path = "../Dataset/"+video_name
    # Set the output directory for the frames
    output_dir = "../Dataset/frames_"+video_name
    # Set the sampling rate (number of frames to skip between extractions)
    sampling_rate = 10
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    cap = cv2.VideoCapture(video_path)
    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for frame_idx in range(0, total_frames, sampling_rate):
        # Set the frame index to extract
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        # Read the frame from the video
        ret, frame = cap.read()
        # Check if the frame was successfully read
        if ret:
            output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))
            cv2.imwrite(output_path, frame)
    # Release the video capture object
    cap.release()
    return 

In [148]:
# TODO: create classes 
"""
desc: Difference between pixel hue,saturation,luma,edges values of adjacent frames.
"""
from typing import Optional
@dataclass
class delta_feature:
    # by default set all weights to 1
    delta_hue: float = 1.0
    delta_sat: float = 1.0
    delta_lum: float = 1.0
    # TODO: have bigger values that other features,detection threshold may need to be adjusted
    delta_edges: float = 1.0  

"""
desc: features calculated for each frame
"""
@dataclass
class frame_data:
    hue: np.ndarray
    sat: np.ndarray
    lum: np.ndarray
    edges: Optional[np.ndarray]

# create defualt weights 
default_delta_feature_weights = delta_feature()

@dataclass
class shot_detector:
    threshold: float = 27.0
    min_scene_len: int = 15
    weights: 'delta_feature' = default_delta_feature_weights
    kernel_size: Optional[int] = None
    last_scene_cut: Optional[int] = None
    last_frame: Optional[frame_data] = None
    frame_score: float=None


In [149]:
# declare object from class 
video_try = shot_detector()
print(video_try.weights)
frame_score: float = (
            sum(getattr(video_try.weights, field1.name)* getattr(video_try.weights, field2.name) for (field1, field2) in zip(fields(video_try.weights),fields(video_try.weights)))
            / sum(abs(getattr(video_try.weights, field3.name)) for field3 in fields(video_try.weights)))
print(frame_score)

delta_feature(delta_hue=1.0, delta_sat=1.0, delta_lum=1.0, delta_edges=1.0)
1.0


In [150]:
"""Detect edges  in the frame 
    params:
        lum:  the luma channel of a frame.
        kernel: kernel size 
    return:
        2D 8-bit image where 255--> edge , 0--> other 
"""
import math

def detect_edges(lum: np.ndarray,kernel:int =None) -> np.ndarray:
    
    if kernel == None:
        # calculate kernel size  depend on the video reselution
        kernel_size = 4 + round(math.sqrt(lum.shape[1]*lum.shape[0]) / 192)
        if kernel_size % 2 == 0:
            kernel_size += 1
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        
    # Estimate levels for thresholding.
    sigma: float = 1.0 / 3.0
    median = np.median(lum)
    low = int(max(0, (1.0 - sigma) * median))
    high = int(min(255, (1.0 + sigma) * median))

    # Calculate edges using Canny algorithm, and reduce noise by dilating the edges.
    edges = cv2.Canny(lum, low, high)
    return cv2.dilate(edges,kernel)

In [151]:
"""
calculate fram score , to compare with the threshold and know if it is shot boundry or not 
params
 frame_num: index of the frame 
 frame_img: frame itself
"""
def calculate_frame_score(video_param:shot_detector,frame_img: np.ndarray) -> float:
        
        # TODO: Add option to enable motion estimation
        
        # Convert image into HSV colorspace
        hue, sat, lum = cv2.split(cv2.cvtColor(frame_img, cv2.COLOR_BGR2HSV))

        # Only calculate edges if we have to
        calculate_edges: bool = (video_param.weights.delta_edges > 0.0)

        edges = detect_edges(lum) if calculate_edges else None

        if video_param.last_frame is None:
            # Need another frame to compare with for score calculation.
            video_param.last_frame = frame_data(hue, sat, lum, edges)
            return 0.0

        score_components = delta_feature(
            delta_hue=mean_pixel_distance(hue, video_param.last_frame.hue),
            delta_sat=mean_pixel_distance(sat, video_param.last_frame.sat),
            delta_lum=mean_pixel_distance(lum, video_param.last_frame.lum),
            delta_edges=(0.0 if edges is None else mean_pixel_distance(
                edges, video_param.last_frame.edges)),
        )
        
        #weights = [video_param.weights.delta_hue,video_param.weights.delta_sat,video_param.weights.delta_lum,video_param.weights.delta_edges]
        # calculate score 
        frame_score: float = (
            sum(getattr(score_components, sco_com.name)* getattr(video_try.weights, wei.name) for (sco_com, wei) in zip(fields(score_components),fields(video_try.weights)))
            / sum(abs(getattr(video_try.weights, weigh.name)) for weigh in fields(video_try.weights)))

        # Store all data required to calculate the next frame's score.
        video_param.last_frame = frame_data(hue, sat, lum, edges)
        return frame_score

In [152]:
"""
return: List of frames where scene cuts have been detected
"""
def process_frame(video_param:shot_detector,frame_num: int, frame_img: np.ndarray):
        if frame_img is None:
            return []

        # initialize the beginning of the frames that contain shot boundery
        if video_param.last_scene_cut is None:
            video_param.last_scene_cut = frame_num

        video_param.frame_score = calculate_frame_score(video_param,frame_img)
        if video_param.frame_score is None:
            return []

        # consider any frame over the threshold a new scene, but only if
        # the minimum scene length has been reached (otherwise it is ignored).
        if video_param.frame_score >= video_param.threshold and (
            (frame_num - video_param.last_scene_cut) >= video_param.min_scene_len):
            video_param.last_scene_cut = frame_num

            # TODO: create average feature vector for these frames 
            

            return [frame_num]

        return []

In [153]:

# try 
import cv2
import os

# Set the video file path
video_path = "try.mp4"

# Set the output directory for the frames
output_dir = "frames"

# Set the sampling rate (number of frames to skip between extractions)
sampling_rate = 10

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Open the video file
cap = cv2.VideoCapture(video_path)

# declare object from class 
video_try = shot_detector()

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop through the frames and extract the sampled frames
for frame_idx in range(0, total_frames, sampling_rate):
    # Set the frame index to extract
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

    # Read the frame from the video
    ret, frame = cap.read()

    # Check if the frame was successfully read
    if ret:
        # Construct the output file path for the frame
        output_path = os.path.join(output_dir, "frame_{:06d}.jpg".format(frame_idx))

        # Write the frame to the output file
        cv2.imwrite(output_path, frame)
    print(process_frame(video_try,frame_idx,frame))
    
# Release the video capture object
cap.release()

hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[]
hue::::::
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 .

In [154]:
"""
get the average feature vector to represent shot with it's frames features 

params:
 List: array of the shot cut frames number
Return:
 average feature vector represent each frame   
"""
def get_avg_features_shot(shot_cuts:np.array):
    
    return 


In [155]:
# # TODO: get_HSV_feature_frame
# """
# params : frames's path
# return : HSV hist for all frames
# """
# def get_HSV_feature_frame():
#     # Set the number of histogram bins
#     num_bins = 16
#     HSV_hists = []
#     frames_path = 'Dataset/frames'
#     # Loop through the frames and extract the color histograms
#     for fram_name in os.listdir(frames_path):

#         frame = cv2.imread(os.path.join(frames_path, fram_name))
#         # Convert the frame to the HSV color space
#         hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#         # Compute the color histogram for each channel
#         hist_hue = cv2.calcHist([hsv_frame], [0], None, [num_bins], [0, 180])
#         hist_saturation = cv2.calcHist([hsv_frame], [1], None, [num_bins], [0, 255])
#         hist_value = cv2.calcHist([hsv_frame], [2], None, [num_bins], [0, 255])

#         # Concatenate the histograms into a single feature vector
#         hist = np.concatenate([hist_hue, hist_saturation, hist_value], axis=0)

#         # Normalize the histogram to have unit L2 norm
#         hist = cv2.normalize(hist, hist)
#         HSV_hists.append(hist)
#     return 

In [156]:
# # TODO: Optical flow (motion pattern feature)

# # read the first frame 
# frames_path = '../Dataset/frames'

# frame1 = cv2.imread(os.path.join(frames_path,os.listdir(frames_path)[0]))
# prvs_fram = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
# threshold = 1000
# shot_boundaries =[]
# for fram_name in os.listdir(frames_path)[1:]:
    
#     next_fram = cv2.imread(os.path.join(frames_path, fram_name))
    
#     gray_next_frame = cv2.cvtColor(next_fram, cv2.COLOR_BGR2GRAY)
    
#     # Calculate optical flow using Farneback algorithm
    
#     #TODO: these parameters may be changed later
#     flow = cv2.calcOpticalFlowFarneback(prvs_fram,gray_next_frame,flow=None,pyr_scale=0.5,levels=3,winsize=15,iterations=3
#                                         ,poly_n=5,poly_sigma=1.2,flags=0)
    
#     # convert the x,y components of the flow vector to magnitude and angle 
#     mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1])
    
#     # Compute the average motion magnitude for the frame
#     avg_mag = cv2.mean(mag)[0]

#     # Check if the average motion magnitude exceeds the threshold
#     if avg_mag > threshold:
#         shot_boundaries.append(cap.get(cv2.CAP_PROP_POS_FRAMES))
    
#     # Update the previous frame
#     prvs_fram = gray_next_frame
# print(shot_boundaries)
       